# 第2章 商品推薦を実現する数理

このNotebookでは、レコメンドシステムを実現するための数理的基礎を学びます。

**キーワード**：協調フィルタリング、行列因子分解、最小二乗法、勾配降下法、評価値行列、コサイン類似度、指示関数、損失関数、内積、三角関数、ベクトル、微分、偏微分、行列

## 目次
- 2-1 はじめに
- 2-2 商品の評価を数理的に表現する ー評価値行列ー
- 2-3 評価値の予測を数理モデルで実現する ー協調フィルタリングと行列因子分解ー
- 2-4 ユーザー同士の類似度で予測値を推計する ー内積の定理とコサイン類似度ー
- 2-5 コサイン類似度の意味を考える ー三角関数ー
- 2-6 コサイン類似度を複数のアイテムに適用する ー多次元への拡張ー

## 環境設定

In [ ]:
# 日本語フォントのインストール（Google Colab用）
!pip install japanize-matplotlib -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの設定
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

---
# 2-1 はじめに

私たちの生活は**レコメンド**に溢れています。

- オンラインショッピングの「おすすめの商品」
- YouTubeの「あなたへのおすすめ動画」
- SNSの「興味のある投稿」

レコメンドは、レコメンドのために作られた**数理モデル**が膨大なデータを処理することで実現します。

本章では、レコメンドを実現する基礎的な数学的手法に焦点を当てて解説します。具体的には、**協調フィルタリング**と**行列因子分解**と呼ばれる数理的手法を中心に解説し、いかにして数理モデルが私たちの「好み」を理解し、購入する可能性が高い商品やコンテンツをレコメンドしているのかを解き明かします。

---
# 2-2 商品の評価を数理的に表現する ー評価値行列ー

レコメンドを数理モデルで実現するために、まず**評価値行列**を作成します。

## ベクトルによる表現

ベクトルは数値の順序付きリストです。

$$\vec{r} = \begin{pmatrix} r_1 \\ r_2 \\ \vdots \\ r_n \end{pmatrix}, \quad \vec{r}^T = (r_1, r_2, \cdots, r_n)$$

- 左側の式は縦の「列方向」に数値が並んでいるので**列ベクトル**
- 右側の式は横の「行方向」に数値が並んでいるので**行ベクトル**
- $^T$ は**転置**を表す記号で、行と列の方向を変える操作を指す

In [ ]:
# ユーザーごとの評価値をベクトルで表現
# 図2.3の例：4人のユーザーが4つのアイテムを評価
# 0は欠損値（未評価）を表す

# 行ベクトル表現
R_u1 = np.array([2, 3, 0, 5])  # user1の評価
R_u2 = np.array([2, 5, 0, 5])  # user2の評価
R_u3 = np.array([0, 3, 4, 4])  # user3の評価
R_u4 = np.array([4, 2, 3, 0])  # user4の評価

print("=== ユーザーごとの評価ベクトル（行ベクトル） ===")
print(f"R_u1 = {R_u1}")
print(f"R_u2 = {R_u2}")
print(f"R_u3 = {R_u3}")
print(f"R_u4 = {R_u4}")

In [ ]:
# 列ベクトル表現（アイテムごとの評価）
R_i1 = np.array([[2], [2], [0], [4]])  # item1の評価
R_i2 = np.array([[3], [5], [3], [2]])  # item2の評価
R_i3 = np.array([[0], [0], [4], [3]])  # item3の評価
R_i4 = np.array([[5], [5], [4], [0]])  # item4の評価

print("=== アイテムごとの評価ベクトル（列ベクトル） ===")
print(f"R_i1 = {R_i1.flatten()}")
print(f"R_i2 = {R_i2.flatten()}")
print(f"R_i3 = {R_i3.flatten()}")
print(f"R_i4 = {R_i4.flatten()}")

## 評価値行列 R

行ベクトル $R_{u_1} \sim R_{u_4}$ を列方向（縦）に並べると、評価値を1つにまとめた**行列**で表現できます。これを**評価値行列**と呼び、$R$と表記します。

$$R = \begin{pmatrix} R_{u_1} \\ R_{u_2} \\ R_{u_3} \\ R_{u_4} \end{pmatrix} = \begin{pmatrix} 2 & 3 & 0 & 5 \\ 2 & 5 & 0 & 5 \\ 0 & 3 & 4 & 4 \\ 4 & 2 & 3 & 0 \end{pmatrix}$$

- $R$ は Rating（評価）の頭文字
- 0 は未評価、つまり**欠損値**を示す
- user1のitem1に対する評価値を $r_{1,1}$ と表記する

一般化すると、ユーザーの総数を $U$、アイテムの総数を $I$ とすると：

$$R = \begin{pmatrix} r_{1,1} & r_{1,2} & \cdots & r_{1,I} \\ r_{2,1} & r_{2,2} & \cdots & r_{2,I} \\ \vdots & \vdots & \ddots & \vdots \\ r_{U,1} & r_{U,2} & \cdots & r_{U,I} \end{pmatrix}$$

In [ ]:
# 評価値行列の作成
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("=== 評価値行列 R ===")
print(R)
print(f"\n行列のサイズ: {R.shape[0]}ユーザー × {R.shape[1]}アイテム")

In [ ]:
# 評価値行列の可視化
fig, ax = plt.subplots(figsize=(8, 6))

# ヒートマップを作成（0は欠損値として特別扱い）
masked_R = np.ma.masked_where(R == 0, R)
im = ax.imshow(masked_R, cmap='YlOrRd', vmin=1, vmax=5)

# 欠損値（0）をグレーで表示
ax.imshow(np.where(R == 0, 1, np.nan), cmap='gray', vmin=0, vmax=1, alpha=0.3)

# 軸ラベル
ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 各セルに値を表示
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            text = ax.text(j, i, '−', ha='center', va='center', color='gray', fontsize=14)
        else:
            text = ax.text(j, i, R[i, j], ha='center', va='center', color='black', fontsize=14)

ax.set_title('評価値行列 R（グレー: 欠損値）')
plt.colorbar(im, label='評価値')
plt.tight_layout()
plt.show()

---
# 2-3 評価値の予測を数理モデルで実現する ー協調フィルタリングと行列因子分解ー

## レコメンドの前提条件

- ユーザーは、購入したすべての商品に評価値を付与している
- 逆に、ユーザーは購入していない商品に評価値を付与していない
- レコメンドする商品は、そのユーザーが未購入の商品（評価値が欠損している商品）に限る

つまり、**欠損部分の評価値を予測値として算出できれば、「もし購入したらいくらの評価を付けるか」を推定できる**。これがレコメンドの有効な判断指標となります。

In [ ]:
# レコメンドの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：評価値がある部分とない部分
ax1 = axes[0]
colors = np.where(R == 0, 0.3, 0.8)
ax1.imshow(colors, cmap='Blues', vmin=0, vmax=1)
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax1.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
        else:
            ax1.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)
ax1.set_xticks(range(4))
ax1.set_yticks(range(4))
ax1.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax1.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax1.set_title('評価値行列（? = 欠損値 = レコメンド対象）')

# 右：予測値の例
ax2 = axes[1]
R_predicted = R.copy().astype(float)
R_predicted[0, 2] = 4.2  # user1のitem3を予測
R_predicted[1, 2] = 4.5  # user2のitem3を予測
R_predicted[2, 0] = 2.1  # user3のitem1を予測
R_predicted[3, 3] = 3.8  # user4のitem4を予測

im2 = ax2.imshow(R_predicted, cmap='YlOrRd', vmin=1, vmax=5)
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax2.text(j, i, f'{R_predicted[i,j]:.1f}', ha='center', va='center', 
                    fontsize=12, color='blue', fontweight='bold')
        else:
            ax2.text(j, i, str(int(R_predicted[i, j])), ha='center', va='center', fontsize=14)
ax2.set_xticks(range(4))
ax2.set_yticks(range(4))
ax2.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax2.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax2.set_title('予測後（青字 = 予測値）')
plt.colorbar(im2, ax=ax2, label='評価値')

plt.tight_layout()
plt.show()

print("例：user1のitem3の予測値が4.2なら、item3をuser1にレコメンドすべき！")

## 予測値の推計方法：3つのアプローチ

| 手法 | 説明 | 軸 |
|:---|:---|:---|
| ①userを軸とした予測値の算出 | 各itemの評価が似ているuserを選び、そのuserの評価値を用いて予測値を算出 | user |
| ②itemを軸とした予測値の算出 | 各userの評価が似ているitemを選び、そのitemの評価値を用いて予測値を算出 | item |
| ③行列因子分解（MF） | 評価値行列を分解した潜在因子行列を解析的に求めることで、予測値を算出 | − |

①と②は**協調フィルタリング**という手法に分類されます。
③は協調フィルタリングとは異なる手法で、評価値行列を**潜在因子行列**と呼ばれる行列に分解して欠損値を推定するアプローチです。

In [ ]:
# 3つのアプローチの概念図
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# ①userを軸とした予測
ax1 = axes[0]
ax1.text(0.5, 0.9, '①userを軸とした予測値の算出', ha='center', fontsize=11, fontweight='bold', transform=ax1.transAxes)
ax1.text(0.5, 0.7, '似ているユーザーの評価値を利用', ha='center', fontsize=10, transform=ax1.transAxes)
# 簡単な図示
ax1.annotate('', xy=(0.7, 0.4), xytext=(0.3, 0.4),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))
ax1.text(0.2, 0.4, 'user1', fontsize=10, ha='center', va='center')
ax1.text(0.8, 0.4, 'user2\n(類似)', fontsize=10, ha='center', va='center', color='blue')
ax1.text(0.5, 0.2, '→ user2の評価を参考にuser1の欠損を予測', ha='center', fontsize=9, transform=ax1.transAxes)
ax1.axis('off')

# ②itemを軸とした予測
ax2 = axes[1]
ax2.text(0.5, 0.9, '②itemを軸とした予測値の算出', ha='center', fontsize=11, fontweight='bold', transform=ax2.transAxes)
ax2.text(0.5, 0.7, '似ているアイテムの評価値を利用', ha='center', fontsize=10, transform=ax2.transAxes)
ax2.annotate('', xy=(0.7, 0.4), xytext=(0.3, 0.4),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax2.text(0.2, 0.4, 'item1', fontsize=10, ha='center', va='center')
ax2.text(0.8, 0.4, 'item2\n(類似)', fontsize=10, ha='center', va='center', color='green')
ax2.text(0.5, 0.2, '→ item2の評価を参考にitem1の欠損を予測', ha='center', fontsize=9, transform=ax2.transAxes)
ax2.axis('off')

# ③行列因子分解
ax3 = axes[2]
ax3.text(0.5, 0.9, '③行列因子分解（MF）', ha='center', fontsize=11, fontweight='bold', transform=ax3.transAxes)
ax3.text(0.5, 0.7, 'R ≈ P × Q', ha='center', fontsize=12, transform=ax3.transAxes)
ax3.text(0.15, 0.4, 'R', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax3.text(0.32, 0.4, '≈', fontsize=14, ha='center')
ax3.text(0.5, 0.4, 'P', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax3.text(0.62, 0.4, '×', fontsize=14, ha='center')
ax3.text(0.75, 0.4, 'Q', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightyellow'))
ax3.text(0.5, 0.2, '→ 潜在因子行列に分解して欠損を推定', ha='center', fontsize=9, transform=ax3.transAxes)
ax3.axis('off')

plt.tight_layout()
plt.show()

---
# 2-4 ユーザー同士の類似度で予測値を推計する ー内積の定理とコサイン類似度ー

まずは「①userを軸とした予測値の算出」について解説します。

この手法では、各itemの評価が似ているuserを選び、そのuserの評価値を用いて予測値を算出します。
このとき、「各itemの評価が似ているuserを選ぶ」という処理を実現するために、**コサイン類似度**という手法を採用します。

## 基礎となる数理的手法と情報工学的アプローチ

| 基礎となる数理的手法 | 情報工学的アプローチ |
|:---|:---|
| 内積の定理：$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta$ | コサイン類似度：$\cos(a, b) = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}}$ |
| ベクトルの大きさ（三平方の定理）：$|\vec{a}| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2}$ | |

## 座標空間での表現

例として、userが6人、itemが2個だけの状況を想定し、以下の評価値行列が作成されたとしましょう。

$$R = \begin{pmatrix} 2 & 3 \\ 2 & 5 \\ 0 & 3 \\ 4 & 2 \\ 4 & 4 \\ 3 & 0 \end{pmatrix}$$

この評価値行列を**座標空間**で示すと、ユーザー同士の類似度合いが視覚的に把握できます。

In [ ]:
# 6人のユーザー、2つのアイテム
R_simple = np.array([
    [2, 3],  # user1
    [2, 5],  # user2
    [0, 3],  # user3 (item1未評価)
    [4, 2],  # user4
    [4, 4],  # user5
    [3, 0]   # user6 (item2未評価)
])

print("評価値行列 R:")
print(R_simple)

In [ ]:
# 図2.10: 座標空間でのプロット
fig, ax = plt.subplots(figsize=(8, 8))

users = ['user1', 'user2', 'user3', 'user4', 'user5', 'user6']
colors = ['blue', 'blue', 'gray', 'blue', 'blue', 'gray']

for i, (user, color) in enumerate(zip(users, colors)):
    x, y = R_simple[i]
    if x == 0 or y == 0:
        # 欠損値があるユーザー
        ax.scatter(x, y, s=100, c='gray', alpha=0.5, zorder=5)
        ax.annotate(user, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10, color='gray')
    else:
        ax.scatter(x, y, s=100, c='blue', zorder=5)
        ax.annotate(user, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_xlabel('item1への評価値', fontsize=12)
ax.set_ylabel('item2への評価値', fontsize=12)
ax.set_title('図2.10: 座標空間でのユーザー分布', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

# 評価なしの軸を示す
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.text(-0.3, 0, '評価\nなし', fontsize=9, va='center', color='gray')
ax.text(0, -0.3, '評価なし', fontsize=9, ha='center', color='gray')

plt.tight_layout()
plt.show()

## ベクトルによる表現

ここで、目的を「user1に似ているユーザーが誰かを特定すること」とし、コサイン類似度によって類似ユーザーを特定します。

考えやすいように、考察対象をuser1, user2, user4に絞り、各userをベクトルで表現します。

$$\vec{u}_1 = (2, 3), \quad \vec{u}_2 = (2, 5), \quad \vec{u}_4 = (4, 2)$$

In [ ]:
# 図2.11: ベクトルとしての表現
fig, ax = plt.subplots(figsize=(8, 8))

# user1, user2, user4のベクトル
u1 = np.array([2, 3])
u2 = np.array([2, 5])
u4 = np.array([4, 2])

# ベクトルを矢印で描画
ax.quiver(0, 0, u1[0], u1[1], angles='xy', scale_units='xy', scale=1, 
          color='blue', width=0.02, label=f'$\\vec{{u}}_1$ = (2, 3)')
ax.quiver(0, 0, u2[0], u2[1], angles='xy', scale_units='xy', scale=1, 
          color='green', width=0.02, label=f'$\\vec{{u}}_2$ = (2, 5)')
ax.quiver(0, 0, u4[0], u4[1], angles='xy', scale_units='xy', scale=1, 
          color='red', width=0.02, label=f'$\\vec{{u}}_4$ = (4, 2)')

# 点も表示
ax.plot(u1[0], u1[1], 'bo', markersize=8)
ax.plot(u2[0], u2[1], 'go', markersize=8)
ax.plot(u4[0], u4[1], 'ro', markersize=8)

# ラベル
ax.annotate(f'({u1[0]},{u1[1]})\nuser1', (u1[0], u1[1]), xytext=(10, 5), 
            textcoords='offset points', fontsize=10)
ax.annotate(f'({u2[0]},{u2[1]})\nuser2', (u2[0], u2[1]), xytext=(10, 5), 
            textcoords='offset points', fontsize=10)
ax.annotate(f'({u4[0]},{u4[1]})\nuser4', (u4[0], u4[1]), xytext=(10, -15), 
            textcoords='offset points', fontsize=10)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_xlabel('item1', fontsize=12)
ax.set_ylabel('item2', fontsize=12)
ax.set_title('図2.11: user1, user2, user4をベクトルで表現', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.legend(loc='lower right')
ax.text(0, -0.3, '評価なし', fontsize=9, ha='center', color='gray')

plt.tight_layout()
plt.show()

print("図から、user1に似ているのはuser2だと言えそうです。")
print("なぜなら、ベクトルを示す矢印のuser1とuser2が作る角度がuser1とuser4が作る角度よりも小さいからです。")

## 内積の定理

角度を「目」で見て類似度を評価することはできますが、コンピュータは「目」で判断することはできず、**数値**でしか判断できません。

そこで、この角度を**定量化**します。このときに便利な数理的手法が高校数学で学習した**内積の定理**です。

2つのベクトル $\vec{a}$, $\vec{b}$ があり、$\vec{a}$, $\vec{b}$ のなす角を $\theta$（theta：シータ）とします。さらに、$\vec{a}$, $\vec{b}$ のそれぞれのベクトルの大きさを $|\vec{a}|$, $|\vec{b}|$ と表現します。

すると、**内積**は式 (2-3) のように定義されます。

$$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta \quad (2\text{-}3)$$

In [ ]:
# 図2.13: 内積の定理を考えるための図
fig, ax = plt.subplots(figsize=(8, 6))

# 2つのベクトル
a = np.array([4, 0])
b = np.array([3, 2])

# ベクトルを描画
ax.quiver(0, 0, a[0], a[1], angles='xy', scale_units='xy', scale=1, 
          color='blue', width=0.02)
ax.quiver(0, 0, b[0], b[1], angles='xy', scale_units='xy', scale=1, 
          color='red', width=0.02)

# ラベル
ax.annotate('$\\vec{a}$', (a[0]/2, a[1]-0.3), fontsize=14, color='blue')
ax.annotate('$\\vec{b}$', (b[0]/2-0.3, b[1]/2+0.2), fontsize=14, color='red')
ax.annotate('$|\\vec{a}|$', (a[0]/2, -0.5), fontsize=12, ha='center')
ax.annotate('$|\\vec{b}|$', (b[0]+0.3, b[1]/2), fontsize=12)

# 角度θを示す弧
theta_arc = np.linspace(0, np.arctan2(b[1], b[0]), 30)
arc_r = 0.8
ax.plot(arc_r * np.cos(theta_arc), arc_r * np.sin(theta_arc), 'g-', linewidth=2)
ax.text(1.0, 0.3, '$\\theta$', fontsize=14, color='green')

ax.set_xlim(-0.5, 5)
ax.set_ylim(-1, 3)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.grid(True, alpha=0.3)
ax.set_title('図2.13: 内積の定理を考えるために、2つのベクトル $\\vec{a}$, $\\vec{b}$ を考える', fontsize=12)

plt.tight_layout()
plt.show()

print("内積の定理: $\\vec{a} \\cdot \\vec{b} = |\\vec{a}||\\vec{b}|\\cos\\theta$")

## cos θ の計算

内積には角度 $\theta$ を変数とした $\cos\theta$ が表れます。よって、式 (2-3) の両辺を $|\vec{a}||\vec{b}|$ で割り、式 (2-4) のように変形することで、$\cos\theta$ の値が計算できるようになります。

$$\cos\theta = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} \quad (2\text{-}4)$$

### 分子：内積 $\vec{a} \cdot \vec{b}$ の計算

2つのベクトル $\vec{a} = (a_1, a_2)$, $\vec{b} = (b_1, b_2)$ について、内積は式 (2-6) のように計算されます。

$$\vec{a} \cdot \vec{b} = a_1 b_1 + a_2 b_2 \quad (2\text{-}6)$$

### 分母：ベクトルの大きさ（ノルム）の計算

ベクトルの大きさは**三平方の定理**を用います。

$$|\vec{a}| = \sqrt{a_1^2 + a_2^2} \quad (2\text{-}10)$$

In [ ]:
# コサイン類似度の計算
def cosine_similarity(a, b):
    """
    2つのベクトルのコサイン類似度を計算する
    
    式: cos(a, b) = (a · b) / (|a| × |b|)
    """
    a = np.array(a)
    b = np.array(b)
    
    # 内積（分子）
    dot_product = np.dot(a, b)
    
    # ノルム（分母）
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    # ゼロベクトルのチェック
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot_product / (norm_a * norm_b)

print("cosine_similarity関数を定義しました")

In [ ]:
# user1, user2, user4のコサイン類似度を計算
u1 = np.array([2, 3])
u2 = np.array([2, 5])
u4 = np.array([4, 2])

print("=== コサイン類似度の計算 ===")
print(f"$\\vec{{u}}_1$ = {tuple(u1)}")
print(f"$\\vec{{u}}_2$ = {tuple(u2)}")
print(f"$\\vec{{u}}_4$ = {tuple(u4)}")
print()

# user1とuser2
print("--- cos(u1, u2) の計算 ---")
dot_12 = np.dot(u1, u2)
norm_1 = np.linalg.norm(u1)
norm_2 = np.linalg.norm(u2)
print(f"$\\vec{{u}}_1 \\cdot \\vec{{u}}_2$ = 2×2 + 3×5 = {dot_12}")
print(f"|$\\vec{{u}}_1$| = √(2² + 3²) = √{2**2 + 3**2} = {norm_1:.4f}")
print(f"|$\\vec{{u}}_2$| = √(2² + 5²) = √{2**2 + 5**2} = {norm_2:.4f}")
cos_12 = cosine_similarity(u1, u2)
print(f"cos(u1, u2) = {dot_12} / ({norm_1:.4f} × {norm_2:.4f}) ≒ {cos_12:.3f}")
print()

# user1とuser4
print("--- cos(u1, u4) の計算 ---")
dot_14 = np.dot(u1, u4)
norm_4 = np.linalg.norm(u4)
print(f"$\\vec{{u}}_1 \\cdot \\vec{{u}}_4$ = 2×4 + 3×2 = {dot_14}")
print(f"|$\\vec{{u}}_4$| = √(4² + 2²) = √{4**2 + 2**2} = {norm_4:.4f}")
cos_14 = cosine_similarity(u1, u4)
print(f"cos(u1, u4) = {dot_14} / ({norm_1:.4f} × {norm_4:.4f}) ≒ {cos_14:.3f}")

In [ ]:
# 図2.15: コサイン類似度の比較
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: user1とuser2（角度小 = 類似度高）
ax1 = axes[0]
ax1.quiver(0, 0, u1[0], u1[1], angles='xy', scale_units='xy', scale=1, 
           color='blue', width=0.015, label='user1')
ax1.quiver(0, 0, u2[0], u2[1], angles='xy', scale_units='xy', scale=1, 
           color='green', width=0.015, label='user2')
ax1.annotate(f'({u1[0]},{u1[1]})\nuser1', (u1[0], u1[1]), xytext=(5, 5), textcoords='offset points')
ax1.annotate(f'({u2[0]},{u2[1]})\nuser2', (u2[0], u2[1]), xytext=(5, 5), textcoords='offset points')
ax1.set_xlim(-0.5, 5.5)
ax1.set_ylim(-0.5, 5.5)
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)
ax1.set_xlabel('item1')
ax1.set_ylabel('item2')
ax1.set_title(f'角度小\ncos(u1, u2) ≒ {cos_12:.3f}', fontsize=12)
ax1.legend()

# 右: user1とuser4（角度大 = 類似度低）
ax2 = axes[1]
ax2.quiver(0, 0, u1[0], u1[1], angles='xy', scale_units='xy', scale=1, 
           color='blue', width=0.015, label='user1')
ax2.quiver(0, 0, u4[0], u4[1], angles='xy', scale_units='xy', scale=1, 
           color='red', width=0.015, label='user4')
ax2.annotate(f'({u1[0]},{u1[1]})\nuser1', (u1[0], u1[1]), xytext=(5, 5), textcoords='offset points')
ax2.annotate(f'({u4[0]},{u4[1]})\nuser4', (u4[0], u4[1]), xytext=(5, -15), textcoords='offset points')
ax2.set_xlim(-0.5, 5.5)
ax2.set_ylim(-0.5, 5.5)
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)
ax2.set_xlabel('item1')
ax2.set_ylabel('item2')
ax2.set_title(f'角度大\ncos(u1, u4) ≒ {cos_14:.3f}', fontsize=12)
ax2.legend()

plt.suptitle('図2.15: コサイン類似度を計算すると、計算結果の値が大きいほど角度が小さくなると思われる', fontsize=11)
plt.tight_layout()
plt.show()

print(f"\n結論: cos(u1, u2) = {cos_12:.3f} > cos(u1, u4) = {cos_14:.3f}")
print("よって、user4よりもuser2のほうがuser1とのコサイン類似度が大きいことがわかりました。")
print("もし、このコサイン類似度が大きいほど「似ている」と考えられるなら、視覚的な判断と一致しますね。")

---
# 2-5 コサイン類似度の意味を考える ー三角関数ー

ここで疑問点が1つ残っています。「コサイン類似度の計算結果が大きいほど、角度が小さく、似ていると言ってよいのか」という疑問です。

なぜなら、コサイン類似度の要となっている $\cos\theta$ について、現時点ではほとんど詳細を把握できていないからです。

## 単位円を用いた cos θ の定義

**単位円**とは原点を中心とした半径1の円です。

- 単位円上に点 $P$ を取り、原点 $O$ を中心に $x$ 軸の正の部分から反時計回りに $\theta$ だけ回転させた線分 $OP$ について
- $P$ の $x$ 座標を $\cos\theta$ と定義します
- $P$ の $y$ 座標は $\sin\theta$ と定義します

In [ ]:
# 図2.16: 単位円での cos θ の定義
fig, ax = plt.subplots(figsize=(8, 8))

# 単位円を描画
theta_circle = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta_circle), np.sin(theta_circle), 'b-', linewidth=2)

# 軸を描画
ax.axhline(y=0, color='k', linewidth=1)
ax.axvline(x=0, color='k', linewidth=1)

# 点Pを示す（θ = 45°の例）
theta_example = np.radians(45)
P_x = np.cos(theta_example)
P_y = np.sin(theta_example)

# 原点からPへの線分
ax.plot([0, P_x], [0, P_y], 'r-', linewidth=2)
ax.plot(P_x, P_y, 'ro', markersize=10)
ax.annotate('P', (P_x, P_y), xytext=(10, 10), textcoords='offset points', fontsize=14)

# cos θ と sin θ を示す点線
ax.plot([P_x, P_x], [0, P_y], 'g--', linewidth=1.5)
ax.plot([0, P_x], [P_y, P_y], 'g--', linewidth=1.5)

# ラベル
ax.annotate('$\\cos\\theta$', (P_x/2, -0.15), fontsize=12, ha='center')
ax.annotate('$\\sin\\theta$', (-0.2, P_y/2), fontsize=12, va='center')

# 角度θを示す弧
arc_theta = np.linspace(0, theta_example, 30)
arc_r = 0.3
ax.plot(arc_r * np.cos(arc_theta), arc_r * np.sin(arc_theta), 'purple', linewidth=2)
ax.text(0.35, 0.15, '$\\theta$', fontsize=14, color='purple')

# 1と-1を示す
ax.plot(1, 0, 'ko', markersize=6)
ax.plot(-1, 0, 'ko', markersize=6)
ax.plot(0, 1, 'ko', markersize=6)
ax.plot(0, -1, 'ko', markersize=6)
ax.text(1.1, 0, '1', fontsize=12)
ax.text(-1.2, 0, '-1', fontsize=12)
ax.text(0.1, 1.1, '1', fontsize=12)
ax.text(0.1, -1.15, '-1', fontsize=12)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_title('図2.16: 単位円上の点Pのx座標を$\\cos\\theta$、y座標を$\\sin\\theta$と定義する', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## θ と cos θ の関係

すると、$0 \leq \theta \leq 180°$ のとき、$\theta$ の値が大きくなるほど $\cos\theta$ の値が小さくなっていく様子がわかります。

さらに、$\cos\theta$ の値の範囲が $-1 \leq \cos\theta \leq 1$ だということもわかります。

In [ ]:
# 図2.17: θが大きくなるほどcos θが小さくなる
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

angles = [30, 60, 90, 150]

for ax, angle in zip(axes, angles):
    theta = np.radians(angle)
    
    # 単位円
    theta_circle = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta_circle), np.sin(theta_circle), 'b-', linewidth=1)
    
    # 軸
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    
    # 点P
    P_x, P_y = np.cos(theta), np.sin(theta)
    ax.plot([0, P_x], [0, P_y], 'r-', linewidth=2)
    ax.plot(P_x, P_y, 'ro', markersize=8)
    ax.annotate('P', (P_x, P_y), xytext=(5, 5), textcoords='offset points', fontsize=11)
    
    # cos θ を示す
    ax.plot([P_x, P_x], [0, P_y], 'g--', linewidth=1)
    ax.plot(P_x, 0, 'go', markersize=6)
    
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.set_title(f'$\\theta$ = {angle}°\n$\\cos\\theta$ = {np.cos(theta):.2f}', fontsize=12)
    ax.grid(True, alpha=0.3)

plt.suptitle('図2.17: 0 < θ < 180°のとき、θの値が大きくなるほどcos θの値は小さくなる', fontsize=12)
plt.tight_layout()
plt.show()

## コサイン類似度の不等式

以上の数学的な観点を踏まえれば、$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta$ と数式を変形させた理由がよくわかると思います。

つまり、(2-4) には以下の**不等式**が成立するのです。

$$-1 \leq \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} \leq 1 \quad (2\text{-}7)$$

だから、$\frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}$ の計算結果が**1に近いほど**、2つのベクトルのなす角 $\theta$ が小さく、2つのベクトルは「**似ている**」と言えるのです。

In [ ]:
# θ と cos θ の関係グラフ
fig, ax = plt.subplots(figsize=(10, 5))

theta_deg = np.linspace(0, 180, 100)
theta_rad = np.radians(theta_deg)
cos_values = np.cos(theta_rad)

ax.plot(theta_deg, cos_values, 'b-', linewidth=2)

# 重要な点をマーク
important_points = [(0, 1, '同じ方向\n(最も類似)'), 
                    (90, 0, '直交\n(無関係)'), 
                    (180, -1, '反対方向\n(最も非類似)')]

for angle, cos_val, label in important_points:
    ax.plot(angle, cos_val, 'ro', markersize=10)
    ax.annotate(f'{label}\n({angle}°, {cos_val})', 
                xy=(angle, cos_val),
                xytext=(angle+15, cos_val),
                fontsize=10,
                arrowprops=dict(arrowstyle='->', color='gray'))

ax.axhline(y=0, color='k', linewidth=0.5, linestyle='--')
ax.fill_between(theta_deg, 0, cos_values, where=(cos_values > 0), alpha=0.3, color='green', label='類似 (cos θ > 0)')
ax.fill_between(theta_deg, 0, cos_values, where=(cos_values < 0), alpha=0.3, color='red', label='非類似 (cos θ < 0)')

ax.set_xlabel('角度 θ (度)', fontsize=12)
ax.set_ylabel('cos θ', fontsize=12)
ax.set_title('角度θとcos θの関係：θが大きくなるほどcos θは小さくなる', fontsize=14)
ax.set_xticks([0, 30, 60, 90, 120, 150, 180])
ax.set_ylim(-1.2, 1.2)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

---
# 2-6 コサイン類似度を複数のアイテムに適用する ー多次元への拡張ー

以上の考察で、コサイン類似度を用いて「ユーザー同士の嗜好性の類似度」を数値化することに成功しました。しかし、よく考えてみると以下の2点について、依然として考察の余地があります。

1. 2つのアイテム評価しか考慮していないこと
2. ベクトルの近さのみに焦点が当てられていること

ECサイトは膨大なアイテム数を保持しているので、3個以上のアイテム数での類似度の計算でもコサイン類似度が使える必要があります。

## n次元への拡張

アイテム数が $n$ 個になった場合、それぞれのベクトルを
$$\vec{a} = (a_1, a_2, a_3, \cdots, a_n), \quad \vec{b} = (b_1, b_2, b_3, \cdots, b_n)$$
と表すことで、内積は式 (2-8) のように計算できます。

$$\vec{a} \cdot \vec{b} = a_1 b_1 + a_2 b_2 + a_3 b_3 + \cdots + a_n b_n \quad (2\text{-}8)$$

さらに式 (2-8) は総和記号 $\sum$ を用いて以下のように簡潔に表記できます。

$$\vec{a} \cdot \vec{b} = \sum_{k=1}^{n} a_k b_k \quad (2\text{-}9)$$

同様に、ベクトルの大きさ（ノルム）も $n$ 次元に拡張できます。

$$|\vec{a}| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} = \sqrt{\sum_{k=1}^{n} a_k^2} \quad (2\text{-}11, 2\text{-}12)$$

In [ ]:
# 図2.19: 次元数と可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 2次元の場合
ax1 = axes[0]
ax1.set_title('2次元の場合', fontsize=12)
a_2d = np.array([3, 2])
ax1.quiver(0, 0, a_2d[0], a_2d[1], angles='xy', scale_units='xy', scale=1, color='blue', width=0.03)
ax1.plot(a_2d[0], a_2d[1], 'bo', markersize=8)
ax1.annotate(f'$\\vec{{a}}$ = ({a_2d[0]}, {a_2d[1]})', (a_2d[0], a_2d[1]), xytext=(10, 5), textcoords='offset points')
ax1.set_xlim(-0.5, 4)
ax1.set_ylim(-0.5, 3)
ax1.set_xlabel('item1')
ax1.set_ylabel('item2')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# 3次元の場合
ax2 = axes[1]
ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax2.set_title('3次元の場合', fontsize=12)
a_3d = np.array([3, 2, 2])
ax2.quiver(0, 0, 0, a_3d[0], a_3d[1], a_3d[2], color='blue', arrow_length_ratio=0.1)
ax2.scatter(a_3d[0], a_3d[1], a_3d[2], color='blue', s=50)
ax2.set_xlabel('item1')
ax2.set_ylabel('item2')
ax2.set_zlabel('item3')
ax2.text(a_3d[0], a_3d[1], a_3d[2]+0.3, f'$\\vec{{a}}$ = ({a_3d[0]}, {a_3d[1]}, {a_3d[2]})')

# 4次元以上の場合
ax3 = axes[2]
ax3.text(0.5, 0.6, '4次元以上の場合', ha='center', fontsize=14, fontweight='bold', transform=ax3.transAxes)
ax3.text(0.5, 0.4, '?', ha='center', fontsize=60, transform=ax3.transAxes)
ax3.text(0.5, 0.15, '視覚的に捉えることはできない', ha='center', fontsize=11, transform=ax3.transAxes)
ax3.axis('off')

plt.suptitle('図2.19: 4次元以上のベクトルの大きさ（ノルム）を "視覚的に" 捉えることはできない', fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# 図2.20: 3次元でのノルムの計算（三平方の定理の応用）
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# ベクトル a = (a1, a2, a3)
a = np.array([3, 2, 2])

# ベクトルを描画
ax.quiver(0, 0, 0, a[0], a[1], a[2], color='blue', arrow_length_ratio=0.1, linewidth=2)
ax.scatter(a[0], a[1], a[2], color='blue', s=100, label=f'A ({a[0]}, {a[1]}, {a[2]})')

# A'点（xy平面への射影）
ax.scatter(a[0], a[1], 0, color='green', s=80, label=f"A' ({a[0]}, {a[1]}, 0)")
ax.plot([0, a[0]], [0, a[1]], [0, 0], 'g--', linewidth=1.5, label=f"$\\sqrt{{{a[0]}^2 + {a[1]}^2}}$")

# 垂直線
ax.plot([a[0], a[0]], [a[1], a[1]], [0, a[2]], 'r--', linewidth=1.5)

# 軸
ax.set_xlabel('item1 ($a_1$)', fontsize=11)
ax.set_ylabel('item2 ($a_2$)', fontsize=11)
ax.set_zlabel('item3 ($a_3$)', fontsize=11)

# ノルムの計算を表示
norm_a = np.linalg.norm(a)
ax.set_title(f'図2.20: 三平方の定理を応用すれば、3次元以上のベクトルの大きさ（ノルム）を算出できる\n'
            f'$|\\vec{{a}}| = \\sqrt{{{a[0]}^2 + {a[1]}^2 + {a[2]}^2}} = \\sqrt{{{a[0]**2 + a[1]**2 + a[2]**2}}} \\approx {norm_a:.2f}$', 
            fontsize=11)

ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

## 一般化されたコサイン類似度の公式

以上より、$n$ 個のアイテムに対する評価値を2つの $n$ 次元ベクトル
$$\vec{a} = (a_1, a_2, a_3, \cdots, a_n), \quad \vec{b} = (b_1, b_2, b_3, \cdots, b_n)$$
と表すことで、コサイン類似度は式 (2-9)、式 (2-12) を用いて式 (2-13) のように表記できます。

$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}} \quad (2\text{-}13)$$

これで3個以上のアイテム数の場合でも、コサイン類似度が使えるようになりました。

In [ ]:
# n次元コサイン類似度の計算例
print("=== n次元コサイン類似度の計算例 ===")
print()

# 5つのアイテムに対する評価（5次元ベクトル）
user_a = np.array([5, 4, 3, 5, 2])  # ユーザーAの評価
user_b = np.array([4, 5, 3, 4, 1])  # ユーザーBの評価（似ている）
user_c = np.array([1, 2, 5, 1, 5])  # ユーザーCの評価（異なる）

print("5つのアイテムに対する評価:")
print(f"  ユーザーA: {user_a}")
print(f"  ユーザーB: {user_b}")
print(f"  ユーザーC: {user_c}")
print()

# コサイン類似度の計算
cos_ab = cosine_similarity(user_a, user_b)
cos_ac = cosine_similarity(user_a, user_c)

print("コサイン類似度:")
print(f"  cos(A, B) = {cos_ab:.4f}  ← 似ている（値が大きい）")
print(f"  cos(A, C) = {cos_ac:.4f}  ← 異なる（値が小さい）")

In [ ]:
# 最初の評価値行列でのユーザー間類似度
print("=== 評価値行列Rでのユーザー間コサイン類似度 ===")
print()

R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("評価値行列 R:")
print(R)
print()

# 注意：欠損値（0）がある場合は、共通して評価しているアイテムのみで計算するのが一般的
# ここでは簡単のため、0も含めて計算

print("ユーザー間のコサイン類似度:")
users = ['user1', 'user2', 'user3', 'user4']
for i in range(4):
    for j in range(i+1, 4):
        sim = cosine_similarity(R[i], R[j])
        print(f"  cos({users[i]}, {users[j]}) = {sim:.4f}")

print()
print("→ user1とuser2の類似度が最も高い（0.9219）")
print("→ user1にitem3をレコメンドする際、user2の評価を参考にできる！")

---
# まとめ

## 本章で学んだこと

### 評価値行列
- ユーザーの評価をベクトルとして表現
- 複数ユーザーの評価を行列 $R$ としてまとめる
- 欠損値（0）= レコメンド対象

### 協調フィルタリング
- 類似ユーザーの評価を使って欠損値を予測
- 「類似性」をコサイン類似度で定量化

### コサイン類似度
$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}}$$

- 値の範囲: $-1 \leq \cos(a, b) \leq 1$
- 1に近いほど「似ている」
- 0に近いほど「無関係」
- -1に近いほど「反対」

### 三角関数との関係
- 単位円上での $\cos\theta$ の定義
- $\theta$ が大きくなると $\cos\theta$ は小さくなる
- これが「角度が小さい = 類似している」の数学的根拠

In [ ]:
# 最終確認：学んだ関数のまとめ
print("=== 本Notebookで実装した関数 ===")
print()
print("1. cosine_similarity(a, b)")
print("   - 2つのベクトルのコサイン類似度を計算")
print("   - 戻り値: -1から1の値")
print()
print("=== 使用例 ===")
a = [5, 4, 3]
b = [4, 5, 2]
print(f"cosine_similarity({a}, {b}) = {cosine_similarity(a, b):.4f}")